### Build model:

### Reducing memory load:
- crop images
- reduce batch size
- reduce number of channels (have 4 available)
- reduce number of base filters
- reduce depth

### working configurations:

#### single channel cropped beyond zeros, batch size 1, depth 4, number of base filters 16:
depth = 4  
filter_num = 16  
model = isensee2017_model(input_shape=(1, 144,144,144), n_base_filters=filter_num, depth=depth, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped beyond zeros, batch size 1, depth 4, number of base filters 16:
depth = 4  
filter_num = 16  
model = isensee2017_model(input_shape=(4, 144,144,144), n_base_filters=filter_num, depth=depth, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")                      

#### 4-channel cropped full size, batch size 1, depth 4, number of base filters 8:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=8, depth=4, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped full size, batch size 1, depth 5, number of base filters 8:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=8, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped full size, batch size 1, depth 5, number of base filters 12:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

#### 4-channel cropped full size, batch size 2, depth 5, number of base filters 6:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=6, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

### out of memory configurations:
#### 4-channel cropped full size, batch size 1, depth 5, number of base filters 16:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")
                      
#### 4-channel cropped full size, batch size 2, depth 5, number of base filters 12:
model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=12, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

### Imports:

In [1]:
from functools import partial

from keras.layers import Input, LeakyReLU, Add, UpSampling3D, Activation, SpatialDropout3D
from keras.engine import Model
from keras.optimizers import Adam

import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.optimizers import Adam

# K.set_image_dim_ordering('th')
K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_first')

try:
    from keras.engine import merge
except ImportError:
    from keras.layers.merge import concatenate

Using TensorFlow backend.


### Calculating metrics:

In [2]:
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))


def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss

In [3]:
def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False):
    """
    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    elif instance_normalization:
        try:
            from keras_contrib.layers.normalization import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number 
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node 
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)

def create_localization_module(input_layer, n_filters):
    convolution1 = create_convolution_block(input_layer, n_filters)
    convolution2 = create_convolution_block(convolution1, n_filters, kernel=(1, 1, 1))
    return convolution2


def create_up_sampling_module(input_layer, n_filters, size=(2, 2, 2)):
    up_sample = UpSampling3D(size=size)(input_layer)
    convolution = create_convolution_block(up_sample, n_filters)
    return convolution


def create_context_module(input_layer, n_level_filters, dropout_rate=0.3, data_format="channels_first"):
    convolution1 = create_convolution_block(input_layer=input_layer, n_filters=n_level_filters)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(convolution1)
    convolution2 = create_convolution_block(input_layer=dropout, n_filters=n_level_filters)
    return convolution2


create_convolution_block = partial(create_convolution_block, activation=LeakyReLU, instance_normalization=True)


def isensee2017_model(input_shape=(4, 128, 128, 128), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid"):
    """
    This function builds a model proposed by Isensee et al. for the BRATS 2017 competition:
    https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/MICCAI_BraTS_2017_proceedings_shortPapers.pdf
    This network is highly similar to the model proposed by Kayalibay et al. "CNN-based Segmentation of Medical
    Imaging Data", 2017: https://arxiv.org/pdf/1701.03056.pdf
    :param input_shape:
    :param n_base_filters:
    :param depth:
    :param dropout_rate:
    :param n_segmentation_levels:
    :param n_labels:
    :param optimizer:
    :param initial_learning_rate:
    :param loss_function:
    :param activation_name:
    :return:
    """
    inputs = Input(input_shape)

    current_layer = inputs
    level_output_layers = list()
    level_filters = list()
    for level_number in range(depth):
        n_level_filters = (2**level_number) * n_base_filters
        level_filters.append(n_level_filters)

        if current_layer is inputs:
            in_conv = create_convolution_block(current_layer, n_level_filters)
        else:
            in_conv = create_convolution_block(current_layer, n_level_filters, strides=(2, 2, 2))

        context_output_layer = create_context_module(in_conv, n_level_filters, dropout_rate=dropout_rate)

        summation_layer = Add()([in_conv, context_output_layer])
        level_output_layers.append(summation_layer)
        current_layer = summation_layer

    segmentation_layers = list()
    for level_number in range(depth - 2, -1, -1):
        up_sampling = create_up_sampling_module(current_layer, level_filters[level_number])
        concatenation_layer = concatenate([level_output_layers[level_number], up_sampling], axis=1)
        localization_output = create_localization_module(concatenation_layer, level_filters[level_number])
        current_layer = localization_output
        if level_number < n_segmentation_levels:
            segmentation_layers.insert(0, create_convolution_block(current_layer, n_filters=n_labels, kernel=(1, 1, 1)))

    output_layer = None
    for level_number in reversed(range(n_segmentation_levels)):
        segmentation_layer = segmentation_layers[level_number]
        if output_layer is None:
            output_layer = segmentation_layer
        else:
            output_layer = Add()([output_layer, segmentation_layer])

        if level_number > 0:
            output_layer = UpSampling3D(size=(2, 2, 2))(output_layer)

    activation_block = Activation(activation_name)(output_layer)

    model = Model(inputs=inputs, outputs=activation_block)
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function)
    return model

In [4]:
# change the number of labels?

model = isensee2017_model(input_shape=(4, 160, 192, 160), n_base_filters=6, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=3, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")

## Make the labels and test train dictionaries:

In [5]:
# from glob import glob
# paths = glob('/Users/etheredgej/Desktop/MICCAI_BraTS17_Data_Training/train/HGG/*/')
# print(paths)

import os
HGG_dir_list = next(os.walk('./HGG/'))[1]
print(len(HGG_dir_list))
LGG_dir_list = next(os.walk('./LGG/'))[1]
print(len(LGG_dir_list))

210
75


In [6]:
completelist = HGG_dir_list + LGG_dir_list
# print(completelist[0:4])
np.random.shuffle(completelist) # shuffles in place
# print(completelist[0:4])

partition={}
train_percentage=0.7
partition['train']=completelist[0:int(len(completelist)*train_percentage)]
partition['test']=completelist[int(len(completelist)*train_percentage):len(completelist)]

labels={}
# HGG=0
# LGG=1
for directory in HGG_dir_list:
    labels[directory]=0
for directory in LGG_dir_list:
    labels[directory]=1

In [7]:
import numpy as np
from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to


def crop_img(img, rtol=1e-8, copy=True, return_slices=False):
    """Crops img as much as possible
    Will crop img, removing as many zero entries as possible
    without touching non-zero entries. Will leave one voxel of
    zero padding around the obtained non-zero area in order to
    avoid sampling issues later on.
    Parameters
    ----------
    img: Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        img to be cropped.
    rtol: float
        relative tolerance (with respect to maximal absolute
        value of the image), under which values are considered
        negligeable and thus croppable.
    copy: boolean
        Specifies whether cropped data is copied or not.
    return_slices: boolean
        If True, the slices that define the cropped image will be returned.
    Returns
    -------
    cropped_img: image
        Cropped version of the input image
    """

    img = check_niimg(img)
    data = img.get_data()
    infinity_norm = max(-data.min(), data.max())
    passes_threshold = np.logical_or(data < -rtol * infinity_norm,
                                     data > rtol * infinity_norm)

    if data.ndim == 4:
        passes_threshold = np.any(passes_threshold, axis=-1)
    coords = np.array(np.where(passes_threshold))
    start = coords.min(axis=1)
    end = coords.max(axis=1) + 1

    # pad with one voxel to avoid resampling problems
    start = np.maximum(start - 1, 0)
    end = np.minimum(end + 1, data.shape[:3])

    slices = [slice(s, e) for s, e in zip(start, end)]

    if return_slices:
        return slices

    return crop_img_to(img, slices, copy=copy)

### Data generator:

In [8]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html

import numpy as np
import keras
import nibabel as nib

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=2, dim=(240,240,155), n_channels=4,
                 n_classes=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.n_channels, *self.dim))
        y = np.empty((self.batch_size, 3, *self.dim))

        # Generate data
        # Decode and load the data
        for i, ID in enumerate(list_IDs_temp):

            img1 = './data/' + ID + '_flair.nii.gz'
            img2 = './data/' + ID + '_t1.nii.gz'
            img3 = './data/' + ID + '_t1ce.nii.gz'
            img4 = './data/' + ID + '_t2.nii.gz'
            img5 = './data/' + ID + '_seg.nii.gz'

            newimage = nib.concat_images([img1, img2, img3, img4, img5])
            cropped = crop_img(newimage)         
            img_array = np.array(cropped.dataobj)
            z = np.rollaxis(img_array, 3, 0)

            padded_image = np.zeros((5,160,192,160))
            padded_image[:z.shape[0],:z.shape[1],:z.shape[2],:z.shape[3]] = z

            a,b,c,d,seg_mask = np.split(padded_image, 5, axis=0)

            images = np.concatenate([a,b,c,d], axis=0)

            # print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

            # split the channels:
            # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))
            seg_mask_1 = np.zeros((1,160,192,160))
            seg_mask_1[seg_mask.astype(int) > 0] = 1
            seg_mask_2 = np.zeros((1,160,192,160))
            seg_mask_2[seg_mask.astype(int) > 1] = 1
            seg_mask_3 = np.zeros((1,160,192,160))
            seg_mask_3[seg_mask.astype(int) > 2] = 1
            seg_mask_3ch = np.concatenate([seg_mask_1,seg_mask_2,seg_mask_3], axis=0).astype(int)

            X[i,] = images
            y[i,] = seg_mask_3ch
            
#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
        return X, y

In [9]:
import numpy as np

from keras.models import Sequential
# from my_classes import DataGenerator

# Parameters
params = {'dim': (160,192,160),
          'batch_size': 2,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': True}

# # Datasets
# partition = # IDs
# labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['test'], labels, **params)

# # Design model
# model = Sequential()
# [...] # Architecture
# model.compile()

# Train model on dataset
# model.fit_generator(generator=training_generator,
#                     validation_data=validation_generator,
#                     use_multiprocessing=True,
#                     workers=4)

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                   epochs=2, 
                   nb_worker=4)

# model.fit_generator(
#         generator=training_generator,
#         samples_per_epoch=2,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         nb_worker=1,
#         validation_steps=800)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<__main__...., validation_data=<__main__...., workers=4, epochs=2)`


Epoch 1/2
99/99 [==============================] - 1419s 14s/step - loss: -0.0494 - val_loss: -0.0492
Epoch 2/2
99/99 [==============================] - 1351s 14s/step - loss: -0.0529 - val_loss: -0.0509


In [24]:
# score = model.evaluate(x_test, y_test, verbose=0)

params = {'dim': (160,192,160),
          'batch_size': 2,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': False}

# Turned shuffle off so that we can match the values in the dictionary to the predictions. 
# This way we can compare the predictions side-by-side with the ground truth.

validation_generator = DataGenerator(partition['test'], labels, **params)

prediction = model.predict_generator(generator=validation_generator)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
prediction

array([[[[[0.41552654, 0.41493458, 0.4124696 , ..., 0.41919717,
           0.43094784, 0.4295037 ],
          [0.40858293, 0.4157323 , 0.41239575, ..., 0.41845614,
           0.43073726, 0.4306152 ],
          [0.40950683, 0.41629744, 0.41056868, ..., 0.41869524,
           0.42416608, 0.4242473 ],
          ...,
          [0.41592473, 0.42047578, 0.41540405, ..., 0.41522887,
           0.41967237, 0.4186701 ],
          [0.41896218, 0.42376134, 0.41869843, ..., 0.41879424,
           0.418882  , 0.41997367],
          [0.4191587 , 0.42400643, 0.41812247, ..., 0.41676584,
           0.41710752, 0.41858327]],

         [[0.41803116, 0.416531  , 0.41233227, ..., 0.4189327 ,
           0.4293493 , 0.4278168 ],
          [0.40800056, 0.41253585, 0.40715748, ..., 0.41285002,
           0.42404744, 0.42477518],
          [0.40934896, 0.41392112, 0.40600622, ..., 0.41420594,
           0.41683966, 0.4181276 ],
          ...,
          [0.4151377 , 0.41767174, 0.41078275, ..., 0.41110864,
    

In [17]:
# # sanity check on the predictions:
# len(prediction)
# prediction[0].shape
# np.unique(prediction)

(3, 160, 192, 160)

In [25]:
# ! mkdir to_categorical_try
# ! mkdir channel_split

mkdir: cannot create directory ‘channel_split’: File exists


In [26]:
import pickle

pickle.dump( partition, open( "./channel_split/partition.pkl", "wb" ) ) # this has the test/train ID matches

# # access the test list:
# testIDlist = partition['test']
# testIDlist

In [33]:
for i in range(len(prediction)):
    pickle.dump( prediction[i], open( "./channel_split/prediction_"+str(i)+".pkl", "wb" ) )

In [28]:
# https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model
# pickle.dump( model, open( "model.pkl", "wb" ) )
model.save_weights('./channel_split/my_model_weights.h5')